In [1]:
import random
import scipy.stats as sp
import matplotlib.pyplot as plt
import numpy as np
import math
from collections import Counter

In [2]:
def random_kid():
    return random.choice(["boy", "girl"])

both_girls = 0
either_girl = 0
older_girl = 0

random.seed(0)
for _ in range(10000):
    younger = random_kid()
    older = random_kid()
    if older == "girl":
        older_girl += 1
    if older == "girl" and younger == "girl":
        both_girls += 1
    if older == "girl" or younger == "girl":
        either_girl += 1
        
print("P(both | older)", both_girls/older_girl)
print("P(both | either)", both_girls/either_girl)

P(both | older) 0.5007089325501317
P(both | either) 0.3311897106109325


Scipy.stats does not seem to have a random function.

In [3]:
def uniform_pdf(x):
    return 1 if x >= 0 and x < 1 else 0

def uniform_cdf(x):
    if x < 0: return 0
    elif x < 1: return x
    else: return 1


In [4]:
fig= plt.figure()
rv = sp.uniform()
x = list(np.arange(-.2,1.2, .01))
#plt.axes([-.5, 1.5, 0, 1.5])
plt.plot(x, rv.pdf(x))
plt.title("PDF")
plt.ylim(ymax=1.1, ymin=-.1)
plt.show()
plt.title("CDF")
plt.ylim(ymax=1.1, ymin=-.1)
plt.plot(x, rv.cdf(x))
plt.show()

In [5]:
def normal_pdf(x, mu=0, sigma=1):
    sqrt_two_pi = math.sqrt(2 * math.pi)
    return(math.exp(-(x-mu) ** 2/2/sigma**2)/(sqrt_two_pi * sigma))

xs = [x /10.0 for x in range(-50, 50)]
plt.plot(xs, [normal_pdf(x, sigma=1) for x in xs], '-', label="mu=0,sigma=1")
plt.plot(xs, [normal_pdf(x, sigma=2) for x in xs], '-', label="mu=0,sigma=2")
plt.plot(xs, [normal_pdf(x, sigma=0.5) for x in xs], '-', label="mu=0,sigma=0.5")
plt.plot(xs, [normal_pdf(x, mu=-1) for x in xs], '-', label="mu=-1,sigma=1")
plt.legend()
plt.title("Various Normal PDFs without scipy")
plt.show()

In [6]:

plt.plot(xs, sp.norm().pdf(xs),'-', label="mu=0,sigma=1")
plt.plot(xs, sp.norm(0,2).pdf(xs),'-', label="mu=0,sigma=2")
plt.plot(xs, sp.norm(0, .5).pdf(xs),'-', label="mu=0,sigma=.5")
plt.plot(xs, sp.norm(-1, 1).pdf(xs),'-', label="mu=-1,sigma=1")
plt.title("Norm PDF with SciPy")
plt.legend()
plt.show()

In [7]:
def normal_cdf(x, mu=0, sigma=1):
    return(1 + math.erf((x-mu)/math.sqrt(2)/sigma))/2

xs = [x / 10.0 for x in range(-50, 50)]
plt.plot(xs, [normal_cdf(x, sigma=1) for x in xs],'-', label='mu=0, sigma=1')
plt.plot(xs, [normal_cdf(x, sigma=2) for x in xs],'-', label='mu=0, sigma=2')
plt.plot(xs, [normal_cdf(x, sigma=0.5) for x in xs],'-', label='mu=0, sigma=0.5')
plt.plot(xs, [normal_cdf(x, mu=-1) for x in xs],'-', label='mu=-1, sigma=1')
plt.legend(loc=4)
plt.title("Various Normal cdfs without SciPy")
plt.show()

In [8]:
plt.plot(xs, sp.norm().cdf(xs),'-', label="mu=0,sigma=1")
plt.plot(xs, sp.norm(0,2).cdf(xs),'-', label="mu=0,sigma=2")
plt.plot(xs, sp.norm(0, .5).cdf(xs),'-', label="mu=0,sigma=.5")
plt.plot(xs, sp.norm(-1, 1).cdf(xs),'-', label="mu=-1,sigma=1")
plt.title("Norm CDF with SciPy")
plt.legend()
plt.show()

In [9]:
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    if mu != 0 or sigma != 1:
        return mu + sigma*inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z, low_p = -10.0, 0
    hi_z, hi_p = 10.0, 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2
        mid_p = normal_cdf(mid_z)
        if mid_p < p:
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            hi_z, hi_p = mid_z, mid_p
        else:
            break

    return mid_z

print(inverse_normal_cdf(.7))

0.5243968963623047


In [10]:
print(sp.norm().ppf(.7))

0.524400512708


In [11]:
def bernoulli_trial(p):
    return 1 if random.random() < p  else 0

def binomial(n, p):
    return sum(bernoulli_trial(p) for _ in range(n))

print(bernoulli_trial(.4))
print(binomial(20, .4))

1
12


In [12]:
print(sp.bernoulli(.4).rvs())
print(sp.binom(20, .4).rvs())

0
9


In [13]:
def make_hist(p, n, num_points):

    data = [binomial(n, p) for _ in range(num_points)]

    histogram = Counter(data)
    plt.bar([x - 0.4 for x in histogram.keys()],
            [v / num_points for v in histogram.values()],
            0.8,
            color='0.75')

    mu = p * n
    sigma = math.sqrt(n * p * (1 - p))

    xs = range(min(data), max(data) + 1)
    ys = [normal_cdf(i + 0.5, mu, sigma) - normal_cdf(i - 0.5, mu, sigma) for i in xs]
    plt.plot(xs,ys)
    plt.title("Binom hist without SciPy")
    plt.show()

make_hist(0.75, 100, 10000)

In [14]:
def make_hist_scipy(p, n, num_points):

    data = sp.binom(n, p).rvs(num_points)

    histogram = Counter(data)
    plt.bar([x - 0.4 for x in histogram.keys()],
            [v / num_points for v in histogram.values()],
            0.8,
            color='0.75')

    mu = p * n
    sigma = math.sqrt(n * p * (1 - p))

    xs = range(min(data), max(data) + 1)
    ys = [sp.norm(mu,sigma).cdf(i+.5) - sp.norm(mu,sigma).cdf(i -.5) for i in xs]
    plt.plot(xs,ys)
    plt.title("Binom Hist with SciPy")
    plt.show()

make_hist_scipy(.75, 100, 10000)